1. a) Build a categorical naïve bayes classifier with the Iris dataset and evaluate the model.


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

X = np.array([pd.qcut(xi, q=4, labels=False, duplicates='drop') for xi in X.T]).T

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

p_probs = np.bincount(y_tr) / len(y_tr)

l_probs = {}
for c in np.unique(y_tr):
    l_probs[c] = [(pd.Series(X_tr[y_tr==c, i]).value_counts() / len(X_tr[y_tr==c, i])) for i in range(X_tr.shape[1])]

def pred(X):
    posteriors = np.zeros((X.shape[0], len(np.unique(y_tr))))
    for c in np.unique(y_tr):
        for i in range(X.shape[1]):
            posteriors[:, c] += np.log(l_probs[c][i].get(X[:, i], 1e-6))
        posteriors[:, c] += np.log(p_probs[c])
    return np.argmax(posteriors, axis=1)

y_prd = pred(X_te)

acc = np.mean(y_prd == y_te)
print(f'Accuracy: {acc}')


Accuracy: 0.5


b) Build Gaussian naïve bayes classifier for the given Breast cancer dataset.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import norm

data = pd.read_csv('Breast_cancer_data.csv')
X = data.drop('diagnosis', axis=1).values
y = data['diagnosis'].values

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42)

summary = {}
for c in np.unique(y_tr):
    summary[c] = [(np.mean(f), np.std(f)) for f in zip(*X_tr[y_tr==c])]

def g_pdf(x, m, s):
    return norm.pdf(x, loc=m, scale=s)

def pred(X):
    posteriors = np.zeros((X.shape[0], len(np.unique(y_tr))))
    for c in np.unique(y_tr):
        for i in range(X.shape[1]):
            mean, std = summary[c][i]
            posteriors[:, c] += np.log(g_pdf(X[:, i], mean, std))
    return np.argmax(posteriors, axis=1)

y_prd = pred(X_te)

acc = np.mean(y_prd == y_te)
print(f'Accuracy: {acc}')


Accuracy: 0.9298245614035088
